In [29]:
!pip install transformers
!pip install nmslib
!pip install tensorboardX
!pip install transformers
import os
import torch
from transformers import AutoTokenizer, AutoModel
import json
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
import glob
import nmslib
import glob
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")


# Generating Code and Docstring embeddings 

## Embeddings for Training data

### Embeddings for Docstrings

Extracted and formatted dataset can be downloaded from this [drive link](https://drive.google.com/file/d/1DBSW_R-vXMXewkv5WgfzPBF8vhoSKrwg/view?usp=sharing)

In [35]:
# extract the downloaded data to the path and change the following paths  
train_path = './data/jsonl/train/*.jsonl'
test_path =  './data/jsonl/test/python_test_0.jsonl'

In [ ]:
# this code generates embeddings for training docstrings 

def find_embs(start,end):

  trian_files = glob.glob(train_path)
  train_data_CLEARTHIS = []
  for each_file in trian_files:
    with open(each_file, 'r') as json_file:
        json_list_CLEARTHIS = list(json_file)
    for json_str in json_list_CLEARTHIS:
        result_CLEARTHIS = json.loads(json_str)
        train_data_CLEARTHIS.append(result_CLEARTHIS)
        
  train_data_CLEARTHIS = train_data_CLEARTHIS [start : end] # start and end are used to take only 100 traning examples and generate embeddings for them
  # we are only taking 100 instances to create embeddings because of the RAM limitations
  # creating embeddigns for more than 100 instances at once crashes the Colab session

  code_train = []
  docstring_train= []

  del json_list_CLEARTHIS,result_CLEARTHIS

  for i in train_data_CLEARTHIS:

    docstring_train.append(i['docstring'])


  # we have deleted variables using 'del' and 'reset selective'
  # we used these methods to stop Colab session from crashing due to RAM issues
  del train_data_CLEARTHIS



  tokens = {'input_ids': [], 'attention_mask': []}

  # the code below generates embeddings for each docstring from a list of all the dosctrings
  for sentence in docstring_train:

      # encode each dpcstring and append to dictionary
      new_tokens = tokenizer.encode_plus(sentence, max_length=100,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  # reformat list of tensors into single tensor
  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  # attentions masks were used to take care of empty space tokens
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
  outputs_CLEARTHIS = model(**tokens)
  embeddings_CLEARTHIS = outputs_CLEARTHIS.last_hidden_state
  del outputs_CLEARTHIS 
  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings_CLEARTHIS.size()).float()
  masked_embeddings = embeddings_CLEARTHIS * mask
  del embeddings_CLEARTHIS
  summed_CLEARTHIS = torch.sum(masked_embeddings, 1)
  summed_mask_CLEARTHIS = torch.clamp(mask.sum(1), min=1e-9)
  train_code_embs = summed_CLEARTHIS / summed_mask_CLEARTHIS
  
  del summed_CLEARTHIS, summed_mask_CLEARTHIS
  %reset_selective -f CLEARTHIS
  return train_code_embs

  
for iteratorr in range(0,2000,100):
  start = iteratorr
  end = start+100
  # we create embeddings for 100 docstrings at a time due to limited RAM
  a_CLEARTHIS = find_embs(start,end)
  # the final embeddings of 100 docstrings are saved to drive as an array 
  # tensors are saved to drive as generated embeddings takes time and we had RAM issues along with colab stopping session after an hour
  torch.save(a_CLEARTHIS, f'../CodeBERT_embeddings/train_embeddings/doc_tensor_{start}.pt')
  %reset_selective -f CLEARTHIS 
  

### Embeddings for Code Snippets

In [32]:
# this code generates embeddings for training code 
# this function has been defined again as we had to keep code as light and memory efficient as possible, because of which we have used different variables in both the functions

def find_embs(start,end):

  trian_files = glob.glob(train_path)
  train_data_CLEARTHIS = []
  for each_file in trian_files:
    with open(each_file, 'r') as json_file:
        json_list_CLEARTHIS = list(json_file)
    for json_str in json_list_CLEARTHIS:
        result_CLEARTHIS = json.loads(json_str)
        train_data_CLEARTHIS.append(result_CLEARTHIS)
        
  train_data_CLEARTHIS = train_data_CLEARTHIS [start : end] # start and end are used to take only 100 traning examples and generate embeddings for them
  # we are only taking 100 instances to create embeddings because of the RAM limitations
  # creating embeddigns for more than 100 instances at once crashes the Colab session

  code_train = []

  del json_list_CLEARTHIS,result_CLEARTHIS

  for i in train_data_CLEARTHIS:

    code_train.append(i['docstring'])


  # we have deleted variables using 'del' and 'reset selective'
  # we used these methods to stop Colab session from crashing due to RAM issues
  del train_data_CLEARTHIS



  tokens = {'input_ids': [], 'attention_mask': []}

  # the code below generates embeddings for each code snippet from a list of all the code snippets
  for sentence in docstring_train:

      # encode each code snippet and append to dictionary
      new_tokens = tokenizer.encode_plus(sentence, max_length=100,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  # reformat list of tensors into single tensor
  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  # attentions masks were used to take care of empty space tokens
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
  outputs_CLEARTHIS = model(**tokens)
  embeddings_CLEARTHIS = outputs_CLEARTHIS.last_hidden_state
  del outputs_CLEARTHIS 
  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings_CLEARTHIS.size()).float()
  masked_embeddings = embeddings_CLEARTHIS * mask
  del embeddings_CLEARTHIS
  summed_CLEARTHIS = torch.sum(masked_embeddings, 1)
  summed_mask_CLEARTHIS = torch.clamp(mask.sum(1), min=1e-9)
  train_code_embs = summed_CLEARTHIS / summed_mask_CLEARTHIS
  
  del summed_CLEARTHIS, summed_mask_CLEARTHIS
  %reset_selective -f CLEARTHIS
  return train_code_embs


## Embeddings for Testing Data

### Embeddings for Docstrings

In [ ]:
# this code generates embeddings for testing docstrings 
# Everything is similar as done for training data, only the variables are different


def find_embs(start,end):

  test_files = glob.glob(test_path)
  test_data_CLEARTHIS = []
  for each_file in test_files:
    with open(each_file, 'r') as json_file:
        json_list_CLEARTHIS = list(json_file)
    for json_str in json_list_CLEARTHIS:
        result_CLEARTHIS = json.loads(json_str)
        test_data_CLEARTHIS.append(result_CLEARTHIS)
        
  test_data_CLEARTHIS = test_data_CLEARTHIS [start : end] # start and end are used to take only 100 traning examples and generate embeddings for them
  # we are only taking 100 instances to create embeddings because of the RAM limitations
  # creating embeddigns for more than 100 instances at once crashes the Colab session

  docstring_test= []

  del json_list_CLEARTHIS,result_CLEARTHIS

  for i in test_data_CLEARTHIS:

    docstring_test.append(i['docstring'])


  # we have deleted variables using 'del' and 'reset selective'
  # we used these methods to stop Colab session from crashing due to RAM issues
  del test_data_CLEARTHIS



  tokens = {'input_ids': [], 'attention_mask': []}

  # the code below generates embeddings for each docstring from a list of all the dosctrings
  for sentence in docstring_test:

      # encode each dpcstring and append to dictionary
      new_tokens = tokenizer.encode_plus(sentence, max_length=100,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  # reformat list of tensors into single tensor
  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  # attentions masks were used to take care of empty space tokens
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
  outputs_CLEARTHIS = model(**tokens)
  embeddings_CLEARTHIS = outputs_CLEARTHIS.last_hidden_state
  del outputs_CLEARTHIS 
  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings_CLEARTHIS.size()).float()
  masked_embeddings = embeddings_CLEARTHIS * mask
  del embeddings_CLEARTHIS
  summed_CLEARTHIS = torch.sum(masked_embeddings, 1)
  summed_mask_CLEARTHIS = torch.clamp(mask.sum(1), min=1e-9)
  test_code_embs = summed_CLEARTHIS / summed_mask_CLEARTHIS
  
  del summed_CLEARTHIS, summed_mask_CLEARTHIS
  %reset_selective -f CLEARTHIS
  return test_code_embs

  
for iteratorr in range(0,2000,100):
  start = iteratorr
  end = start+100
  # we create embeddings for 100 docstrings at a time due to limited RAM
  a_CLEARTHIS = find_embs(start,end)
  # the final embeddings of 100 docstrings are saved to drive as an array 
  # tensors are saved to drive as generated embeddings takes time and we had RAM issues along with colab stopping session after an hour
  torch.save(a_CLEARTHIS, f'../CodeBERT_embeddings/test_embeddings/doc_tensor_{start}.pt')
  %reset_selective -f CLEARTHIS 
  

### Embeddings for Code snippet

In [ ]:
# this code generates embeddings for testing code 
# Everything is similar as done for training data, only the variables are different


def find_embs(start,end):

  test_files = glob.glob(test_path)
  test_data_CLEARTHIS = []
  for each_file in trian_files:
    with open(each_file, 'r') as json_file:
        json_list_CLEARTHIS = list(json_file)
    for json_str in json_list_CLEARTHIS:
        result_CLEARTHIS = json.loads(json_str)
        test_data_CLEARTHIS.append(result_CLEARTHIS)
        
  test_data_CLEARTHIS = test_data_CLEARTHIS [start : end] # start and end are used to take only 100 traning examples and generate embeddings for them
  # we are only taking 100 instances to create embeddings because of the RAM limitations
  # creating embeddigns for more than 100 instances at once crashes the Colab session

  code_test= []

  del json_list_CLEARTHIS,result_CLEARTHIS

  for i in test_data_CLEARTHIS:

    code_test.append(i['docstring'])


  # we have deleted variables using 'del' and 'reset selective'
  # we used these methods to stop Colab session from crashing due to RAM issues
  del test_data_CLEARTHIS



  tokens = {'input_ids': [], 'attention_mask': []}

  # the code below generates embeddings for each docstring from a list of all the dosctrings
  for sentence in code_test:

      # encode each dpcstring and append to dictionary
      new_tokens = tokenizer.encode_plus(sentence, max_length=100,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  # reformat list of tensors into single tensor
  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  # attentions masks were used to take care of empty space tokens
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
  outputs_CLEARTHIS = model(**tokens)
  embeddings_CLEARTHIS = outputs_CLEARTHIS.last_hidden_state
  del outputs_CLEARTHIS 
  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings_CLEARTHIS.size()).float()
  masked_embeddings = embeddings_CLEARTHIS * mask
  del embeddings_CLEARTHIS
  summed_CLEARTHIS = torch.sum(masked_embeddings, 1)
  summed_mask_CLEARTHIS = torch.clamp(mask.sum(1), min=1e-9)
  test_code_embs = summed_CLEARTHIS / summed_mask_CLEARTHIS
  
  del summed_CLEARTHIS, summed_mask_CLEARTHIS
  %reset_selective -f CLEARTHIS
  return test_code_embs

  
for iteratorr in range(0,2000,100):
  start = iteratorr
  end = start+100
  # we create embeddings for 100 docstrings at a time due to limited RAM
  a_CLEARTHIS = find_embs(start,end)
  # the final embeddings of 100 docstrings are saved to drive as an array 
  # tensors are saved to drive as generated embeddings takes time and we had RAM issues along with colab stopping session after an hour
  torch.save(a_CLEARTHIS, f'../CodeBERT_embeddings/test_embeddings/code_tensor_{start}.pt')
  %reset_selective -f CLEARTHIS 
  

### Combining all the tensors with 100 instances into one main tensor 

In [ ]:
embeddings_code = glob.glob('../CodeBERT_embeddings/train_embeddings/*.pt') # location where tensors with 100 instanes are stored
tensor_list=[]
for emb in embeddings_code:
  x = torch.load(emb)
  tensor_list.append(x)
train_code_embs = torch.cat(tensor_list, dim=0)
torch.save(train_code_embs, f'../CodeBERT_embeddings/train_embeddings/code_doc_main.pt') # change this name according to the experiment for which tensors are being combined
# the array saved in this step will be used for code search

# Fine tuning the codeBERT for code search


In [ ]:
!pip install tensorboardX
!pip install transformers
!pwd
import os
!git clone https://github.com/microsoft/CodeBERT.git
%cd /content/CodeBERT/CodeBERT
!rm -rf data data/codesearch
!mkdir data data/codesearch
%cd data/codesearch
!gdown https://drive.google.com/uc?id=1xgSR34XO8xXZg4cZScDYj2eGerBE9iGo  
!unzip codesearch_data.zip 
!rm  codesearch_data.zip
%cd ../../codesearch

In [ ]:
# following paths are according to the google colab
os.rename('/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/train.txt', '/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/train_o.txt')
os.rename('/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/valid.txt', '/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/valid_o.txt')

In [ ]:
# changing size of input training data to make Code-BERT finetuning faster. 


with open('/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/train_o.txt','r') as f:
  file = f.readlines()
with open('/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/train.txt','w')as f:
  for i in range(0,1000):
    f.write(file[i])
with open('/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/valid_o.txt','r') as f:
  file = f.readlines()
with open('/content/CodeBERT/CodeBERT/data/codesearch/train_valid/python/valid.txt','w')as f:
  for i in range(0,500):
    f.write(file[i])

In [ ]:
!python run_classifier.py \
--model_type roberta \
--task_name codesearch \
--do_train \
--do_eval \
--eval_all_checkpoints \
--train_file train.txt \
--dev_file valid.txt \
--max_seq_length 200 \
--per_gpu_train_batch_size 32 \
--per_gpu_eval_batch_size 32 \
--learning_rate 1e-5 \
--num_train_epochs 8 \
--gradient_accumulation_steps 1 \
--overwrite_output_dir \
--data_dir ../data/codesearch/train_valid/python \
--output_dir ./models/python  \
--model_name_or_path microsoft/codebert-base
